# IPA & Cross Currency Curves
Instrument Pricing Analytics (IPA) is a powerful service provided by LSEG through the Refinitiv Data Platform REST API (more interestingly for us, the [Refinitiv Data Library for Python](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python) (RD)) that provides calculators for financial operations ranging from [Bond Contracts](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-bond-contracts) to [Exchange Traded Instrument (ETI) Volatility Surfaces](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-bond-contracts) and everything inbetween. The functionality we're after in this article is [FX Cross Contracts](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-fx-cross-contracts). Thanks to this service, we can recreate any calculation found in the FWDS app:

![FWDS](../Docs/FWDS.PNG) 

before we go ahead however, it's imperative that we understand what exactly is shown in this app:

## FWDS

There's a lot of information on this FWDS app, some of which can be found on [YouTube](https://www.youtube.com/watch?v=UcDaoknUPUI); let's take it appart topic by topic:

### Forwards

Forwards are simple contracts at heart: it's a contract buying one currency for another in the future at an agreed rate.

One buys a Forward Contract (also known as (a.k.a.): a Forward) to purchase something (*exempli gratia* (e.g.): a currency such as Great British Pounds (GBP)), at a certain price (e.g.: in Euros (EUR)), at a certain date in the future (e.g.: in 1 year's time). It is said that someone is "long FX forward position" when they're exposed to fluctuations in a specific currency through an FX forward contract, e.g.: I am in a long EUR/GBP FX forward position when I bought a forward contract tying me into the purchase of GBP (e.g.: 860GBP) in exchange for sale of EUR (e.g.: 1000EUR) at a rate prescribed by the forward contract ($F_{0,f/d}$) (e.g.: 8.60GBP).

#### Nomeclature: "Base" and "Quote" Currencies

There are, unfortunutally, conventions in FX that complicate things, mainly related to the ["Base Currency" (a.k.a.: "Transaction Currency")](https://www.investopedia.com/terms/b/basecurrency.asp) and the ["Quote Currency"](https://www.investopedia.com/terms/q/quotecurrency.asp). Thankfully, there are a few rules that one may follow to make sense of it all:

1. The Base Currency is always 1. E.g.: You can get 0.07538 CAD (Canadian Dolars) for each MXN (Mexical Pessos), here MXN is the Base Currency, and the CAD is the Quote Currency.
2. The currency closest to the rate is the Quote Currency, and the currency furthest from the rate is the Base Currency. E.g.: "CHFMAD 11.4930" means that the MAD (Moroccan Dirham) is the Quote Currency, and the CHF (Swiss Franc) is Base Currency and you can get 11.4930 MAD per 1 CHF.
3. When an FX rate is quoted without the rate (e.g.: in a calculator), the rate is always assumed to come afterwards. E.g.1: if "ZARFJD" is 0.1177, it represents "ZARFJD 0.1177"; here FJD (Fijian Dollar) is the Quote Currency, and ZAR (South African Rand) is the Base Currency; you can get 0.1177 FJD for every 1 ZAR. E.g.2: if "NOKRON=" is 0.4189, it represents "NOKRON=0.4189"; NOK (Norwegian Krone) is the Base Currency, RON (Romanian Leu) the Quote Currency.

Note that you can get any syymbols in the midle of your numonic; e.g.1: "KRW/PKR=2.0764" means that KRW (South Korean Won) is the Base Currency, PKR (Pakistani Rupee) is the Quote Currency, and you can get 2.0764PKR for every 1KRW; e.g.2: for "53.83 MGA/INR", MGA (Malagasy Ariary) is the Quote Currency and INR (Indian Rupee) is the Base Currency, and you can get 53.83 MGA per 1 INR.

Let's take a concrete example:

#### "Straight" Forwards vs Forwards

1st, let's look at "Straight" Forwards, i.e.: Forwards for two currencies, with none 'in the middle'; the 'middle currency' usually is the USD, and we'll look at them later. You can find the meaning behind any one item in the FWDS add by hovering over it, so I will not explain every item here. Rather, I will use the labels to refer to the buttons and values:

![labeledFWDS](../Docs/labeledFWDS.png)

The FWDS (FX Forward Calculator) calculator combines 3 legacy Refinitiv apps named (i) Swap Points & Outrights, (ii) Deposite Analysis, and (iii) NDFX.
In the screenshot above, we

(I) picked EURGBP in $\textcolor{red}{a}$ (for the app to compute values for the Euro/Great British Pound currency pair) such that EUR is the Base Currency, and GBP is the Quote Currency;

(II) made sure we were calculating values relating to forward contracts for EUR & GBP with $\textcolor{red}{b}$ &  $\textcolor{red}{c}$, as opposed to 'Rates' (i.e.: Deposit Rates) (more on this later),

(III) picked a Trade Date in $\textcolor{red}{d}$ (a date in the past can be chosen to simulate past trades) and the calculator automatically picked a [Value Date](https://www.investopedia.com/terms/v/valuedate.asp#:~:text=Value%20Date%20in%20Trading&text=Due%20to%20differences%20in%20time,agree%20to%20the%20exchange%20rate.) in $\textcolor{red}{e}$ (the date that the the currencies are handed over from buyer to seller; due to differences in time zones and bank processing delays, the value date for spot trades in foreign currencies is usually set two days after a transaction is agreed on, often depicted as "T+2"),

(IV) can glanced at the Bid (left) and Ask (right) [Spot](https://www.investopedia.com/terms/forex/f/forex-spot-rate.asp) rates for the FX pair we have in mind with $\textcolor{red}{f}$. Here the Bid Spot Rate is 0.8633, the Ask Spot Rate is 0.8637;

(V) glanced at the market data that calculations were made with via $\textcolor{red}{g}$

(VI) picked a reference rate of EUR with $\textcolor{red}{g}$, so that we only see the straight pair (only EUR and GBP with no intermediaries; an intermediary could be USD, for e.g., whereby someone would exchange EUR for USD and USD for GBP if there was noone selling GBP for EUR, but there was someone ready to sell USD for EUR, and someone else to sell GBP for USD).

Now that we've done this, we can see the calculated results under "Standard Periods" (to the left of $\textcolor{red}{t}$). What are we looking at exactly?

(V) In column $\textcolor{red}{u}$, we can see the Forward's period we're after. the last row shows the "2Y" period, i.e.: data about Farwards with a 2 year maturity. Explicit dates related to this maturity can be seen in column $\textcolor{red}{w}$; that last row shows data for Forwards written on Oct 16th 2023 (the day I took the screenshot) and ending 2 years later (Oct 16th 2025), that's to say 731 days later, as shown in column column $\textcolor{red}{x}$. If the maturity date falls on a weekend, a yellow $\textcolor{yellow}{W}$ will show up in column $\textcolor{blue}{v}$; if it falls on a local holiday, it will be a red $\textcolor{red}{H}$; the next working day becomes the defacto maturity date.

Note that:

ON = Over Night. Data on this row reffer to FX Forward contracts expiring 1 business day later; 13 Oct 2023 in this instance. Keep in mind that these contracts are settled T+2.

TN = Tomorrow Night. Data on this row reffer to FX Forward contracts starting in 1 business day and ending in 2. In our example, that translates to 3 days because of the weekend. Keep in mind that these contracts are settled T+2.

SN = Spot Next.

SW = Spot Week.


(VI) Columns $\textcolor{red}{y}$ and $\textcolor{red}{z}$ are... complicated.

"Outrights" are the explicit EUR/GBP rates, i.e.: at a 2Y period FX Forward contract, one would get 0.892095 GBP for 1 EUR. Outrights for contracts from Spot Next (SN) (3rd row from the top in $\textcolor{red}{u}$) onwards are calculated differently to the ones bellow (TN and ON).


##### Spot Next Rates +

"Swap Points", in the left column of $\textcolor{red}{z}$, for contracts from Spot Next (SN) (3rd row from the top in $\textcolor{red}{u}$) onwards are the incremental points above the Spot Rate (that can be seen in $\textcolor{red}{f}$); these points are quoted in 1 / 10 000th, i.e.: 283.9500 Swap Points = 0.028395. Keeping with the 2Y period FX Forward contract example:

$$ \begin{array}{ll}
{EURGBP}_{Ask, Outright, 2Y, "2023-10-12"}
    &= {EURGBP}_{Ask, Spot, "2023-10-12"} + {EURGBP}_{Ask, Swap, 2Year, "2023-10-12"} * \text{percent of a percent} \\
    &= {EURGBP}_{Ask, Spot, "2023-10-12"} + \frac{{EURGBP}_{Ask, Swap, 2Year, "2023-10-12"}}{100 * 100 = 10 000}
\end{array}$$

where:

${EURGBP}_{Ask, Outright, 2Y, "2023-10-12"}$ = EUR/GBP Ask Outright rate for a 2 Year FX Forward on 12 Oct 2023 (seen in $\textcolor{blue}{z}$)

${EURGBP}_{Ask, Spot, "2023-10-12"}$ = EUR/GBP Ask Spot rate on 12 Oct 2023 (seen in $\textcolor{red}{f}$)

${EURGBP}_{Ask, Swap, 2Year, "2023-10-12}$ = EUR/GBP Ask Swap Points for a 2 Year FX Forward on 12 Oct 2023 (seen in $\textcolor{blue}{z}$)



##### Tomorrow Night rates

TN rates, in $\textcolor{blue}{z}$, are calculated differnetly to the above. To make sense of them, I created the excel sheet snapshot below.

${EURGBP}_{Ask, Outright, TN, "2023-10-12"} = {EURGBP}_{Ask, Spot, "2023-10-12"} - \frac{{EURGBP}_{Bid, Swap, TN, "2023-10-12"}}{10 000}$

${EURGBP}_{Bid, Outright, TN, "2023-10-12"} = {EURGBP}_{Bid, Spot, "2023-10-12"} - \frac{{EURGBP}_{Ask, Swap, TN, "2023-10-12"}}{10 000}$


##### Overnight (ON) rates

ON rates, in $\textcolor{blue}{z}$, are calculated differnetly to the above. To make sense of them, I created the excel sheet snapshot below, the SN and ON rates are in a clearer background (grey).


$$ \begin{array}{ll}
{EURGBP}_{Ask, Outright, SN, "2023-10-12"}
    &= {EURGBP}_{Ask, Outright, TN, "2023-10-12"} - \frac{{EURGBP}_{Bid, Swap, ON, "2023-10-12"}}{10 000} \\
    &= {EURGBP}_{Ask, Spot, "2023-10-12"} - \frac{{EURGBP}_{Bid, Swap, TN, "2023-10-12"}}{10 000} - \frac{{EURGBP}_{Bid, Swap, ON, "2023-10-12"}}{10 000} \\
    &= {EURGBP}_{Ask, Spot, "2023-10-12"} - \frac{{EURGBP}_{Bid, Swap, TN, "2023-10-12"} + {EURGBP}_{Bid, Swap, ON, "2023-10-12"}}{10 000}
\end{array}$$

and

$$ \begin{array}{ll}
{EURGBP}_{Bid, Outright, SN, "2023-10-12"}
     &={EURGBP}_{Bid, Outright, TN, "2023-10-12"} - \frac{{EURGBP}_{Ask, Swap, ON, "2023-10-12"}}{10 000} \\
    &= {EURGBP}_{Bid, Spot, "2023-10-12"} - \frac{{EURGBP}_{Ask, Swap, TN, "2023-10-12"}}{10 000} - \frac{{EURGBP}_{Ask, Swap, ON, "2023-10-12"}}{10 000} \\
    &= {EURGBP}_{Bid, Spot, "2023-10-12"} - \frac{{EURGBP}_{Ask, Swap, TN, "2023-10-12"} + {EURGBP}_{Ask, Swap, ON, "2023-10-12"}}{10 000}
\end{array}$$

![labeledFWDSexplained](../Docs/labeledFWDSexplained.png)

### How is this done in Python?

Let's stick to this simple example above for now and see how it can be coded in Python with LSEG's IPA (Instrument Pricing Analytics) (a.k.a.: Quantitative Analytics) service.

We are now going to look into [using PEP 3107](https://stackoverflow.com/questions/2489669/how-do-python-functions-handle-the-types-of-parameters-that-you-pass-in) (and [PEP 484](https://stackoverflow.com/questions/2489669/how-do-python-functions-handle-the-types-of-parameters-that-you-pass-in#:~:text=the%20introduction%20of-,PEP%20484,-which%20introduces%20a)) (and some [decorators](https://realpython.com/primer-on-python-decorators)). In line with PEP, I will also now use [PEP8](https://peps.python.org/pep-0008/) naming conventions.

In [52]:
# pip install --trusted-host files.pythonhosted.org nb_mypy # This line can be used to install `nb_mypy`.

In [53]:
import nb_mypy  # !pip3 install nb_mypy --trusted-host pypi.org # https://pypi.org/project/nb-mypy/ # https://gitlab.tue.nl/jupyter-projects/nb_mypy/-/blob/master/Nb_Mypy.ipynb
%load_ext nb_mypy
%reload_ext nb_mypy
%nb_mypy On
%nb_mypy DebugOff

Version 1.0.5


The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy


In [54]:
import refinitiv.data as rd  # This is LSEG's Data and Analytics' API wrapper, called the Refinitiv Data Library for Python. You can update this library with the comand `!pip install refinitiv-data --upgrade`
from refinitiv.data.content import search  # We will use this Python Class in `rd` to fid the instrument we are after, closest to At The Money.
import refinitiv.data.content.ipa.financial_contracts as rdf  # We're going to need this to use the content layer of the RD library and the calculators of greeks and Impl Volat in Instrument Pricing Analytics (IPA) and Exchange Traded Instruments (ETI)
from refinitiv.data.content.ipa.financial_contracts import cross  # We're going to need thtis to use the content layer of the RD library and the calculations

import numpy as np  # We need `numpy` for mathematical and array manipilations.
import pandas as pd  # We need `pandas` for datafame and array manipilations.
import calendar  # We use `calendar` to identify holidays and maturity dates of intruments of interest.
import pytz  # We use `pytz` to manipulate time values aiding `calendar` library. to import its types, you might need to run `!python3 -m pip install types-pytz`. To get the Type Hints for this library, try `pip install types-pytz`.
import pandas_market_calendars as mcal  # Used to identify holidays. See `https://github.com/rsheftel/pandas_market_calendars/blob/master/examples/usage.ipynb` for info on this market calendar library
from datetime import datetime, date, timedelta, timezone  # We use these to manipulate time values
from dateutil.relativedelta import relativedelta  # We use `relativedelta` to manipulate time values aiding `calendar` library. May also need `pip install types-python-dateutil`.
import requests  # We'll need this to send requests to servers vie a the delivery layer - more on that below. Might also need `pip install types-requests`.

# `plotly` is a library used to render interactive graphs:
import plotly.graph_objects as go
import plotly.express as px  # This is just to see the implied vol graph when that field is available
import matplotlib.pyplot as plt  # We use `matplotlib` to just in case users do not have an environment suited to `plotly`.
from IPython.display import clear_output, display  # We use `clear_output` for users who wish to loop graph production on a regular basis. We'll use this to `display` data (e.g.: pandas data-frames).
from plotly import subplots
import plotly

from typing import Generator, Any, TypedDict
from types import ModuleType  # FrameType, TracebackType

In [55]:
rd.open_session(
    name="platform.rdph",  # name="desktop.workspace4", "platform.rdph"
    config_name="C:/Example.DataLibrary.Python-main/Configuration/refinitiv-data.config.json")

c:\Users\U6082174\AppData\Local\Programs\Python\Python310\lib\site-packages\refinitiv\data\_access_layer\session.py:71:FutureWarning: 
You open a platform session using the default value of the signon_control parameter (signon_control=True).
In future versions of the library (2.x), this default will be changed to False.
If you want to keep the same behavior as today, you will need to set the signon_control parameter to True either in the library configuration file
({'sessions':{'platform':{'your_session_name':{'signon_control':true}}}}) or in your code where you create the Platform Session.
These alternative options are already supported in the current version of the library.


<refinitiv.data.session.Definition object at 0x1933a984fa0 {name='rdph'}>

In [56]:
i_str: str
i_ModType: ModuleType

for i_str, i_ModType in zip(
    ['refinitiv.data', 'numpy', 'pandas', 'pandas_market_calendars' 'pytz', 'requests', 'plotly'],
    [rd, np, pd, mcal, pytz, requests, plotly]):
    print(f"{i_str} used in this code is version {i_ModType.__version__}")

refinitiv.data used in this code is version 1.4.0
numpy used in this code is version 1.23.1
pandas used in this code is version 1.3.5
pandas_market_calendarspytz used in this code is version 4.1.4
requests used in this code is version 2022.1
plotly used in this code is version 2.28.1


#### Layers
There are 3 layers in the Refinitiv Data Library: [Delivery](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/3-Delivery), [Content](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/2-Content) and [Access](https://github.com/LSEG-API-Samples/Example.DataLibrary.Python/tree/main/Examples/1-Access). Below, I will give examples for the Delivery and Content layers. An Access Layer is buing built for the below IPA functionalities, but is not out yet.

#### Endpoints
You can reach the IPA API Family via the Quantitative Analytics set of endpoints (they all start with "https://api.refinitiv.com/data/quantitative-analytics"). 

##### [IPA Financial Contracts: FX Cross Contracts (Financial Contracts Endpoint)](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-fx-cross-contracts)
With that said, what we looked at above did not nessesitate calculations, but only data. This is because we're not looking at Cross Currency Curves, Credit Curves or Inflation Curves (more on those below). For this reason, we're only going to use the "financial-contracts" endpoint for now (specifically the [IPA Financial Contracts: FX Cross Contracts (Financial Contracts Endpoint)(ipa-financial-contracts-fx-cross-contracts)](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-fx-cross-contracts); if there's not enough information there, you can also look deeper in the documentation on its [API Playground page](https://apidocs.refinitiv.com/Apps/ApiDocs#/details/L2RhdGEvcXVhbnRpdGF0aXZlLWFuYWx5dGljcy92MQ==/L2ZpbmFuY2lhbC1jb250cmFjdHM=/POST/SWAGGER)):

In [57]:
deliv_resp: rd.delivery._data._endpoint_definition.Definition = rd.delivery.endpoint_request.Definition(
    method=rd.delivery.endpoint_request.RequestMethod.POST,
    url="https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts",
    body_parameters={
        "fields": [
            "InstrumentTag", "ValuationDate", "StartDate", "EndDate", "Tenor", "NonAdjustedStartDate", "NonAdjustedEndDate", "StartDateAdjustmentFlag",
            "EndDateAdjustmentFlag", "SpotDecimals", "FxSwapPointScalingFactor", "StrikeDate_BidAsk", "FxSpot_BidMidAsk", "FxSpotCcy1_BidMidAsk",
            "FxSpotCcy2_BidMidAsk", "FxSwapsCcy1_BidMidAsk", "FxSwapsCcy2_BidMidAsk", "FxSwapsCcy1Ccy2_BidMidAsk", "FxSwapsCcy1Ccy2NearLeg_BidMidAsk",
            "FxSwapsCcy1Ccy2FarLeg_BidMidAsk", "FxOutrightCcy1Ccy2_BidMidAsk", "Ccy1SpotDate", "Ccy2SpotDate", "ErrorMessage"],
        "outputs": ["Data", "Statuses", "Headers"],
        "universe": [{
            "instrumentType": "FxCross",
            "instrumentDefinition": {
                "instrumentTag": "standardperiods",
                "fxCrossCode": "EURGBP",
                "fxCrossType": "FxMultileg", # You can also use "FxForward", but that only allows for one leg.
                "legs": [{"tenor": "ON"},
                         {"tenor": "TN"},
                         {"tenor": "SN"},
                         {"tenor": "SW"},
                         {"tenor": "1M"},
                         {"tenor": "6M"},
                         {"tenor": "9M"},
                         {"tenor": "1Y"},
                         {"tenor": "2Y"}
                         ]},
            "pricingParameters": {
                "valuationDate": datetime.now().strftime(format="%Y-%m-%d") # "2023-11-08T14:00:00Z" # datetime.now().strftime(format="%Y-%m-%d")
                }}],
        "marketData": {
            "fxSpots": [
                {"spotDefinition": {
                    "fxCrossCode": "EURGBP",  # default is vs. USD; EURUSD here
                    "source": "FMD"}},
                # {"spotDefinition": {
                #     "fxCrossCode": "EURGBP",
                #     "source": "Composite"}},
                # {"spotDefinition": {
                #     "fxCrossCode": "EURGBP",
                #     "source": "Calculated"}}
                    ],
            # "fxSwapPoints": [
            #     {"curveDefinition": {
            #         "fxCrossCode": "EURGBP",  # default is vs. USD; EURUSD here
            #         "source": "ICAP"},
            #      "curveParameters": {
            #         "twoLayersTurnsCalculation": True,
            #         "excludeOddPeriods": True,
            #         "excludeLongPeriods": True,
            #         "turnsCalibration": "Quarter-ends"}},
            #     {"curveDefinition": {
            #         "fxCrossCode": "EURGBP",
            #         "source": "Composite"},
            #      "curveParameters": {
            #             "twoLayersTurnsCalculation": True,
            #             "excludeOddPeriods": True,
            #             "excludeLongPeriods": True,
            #             "turnsCalibration": "Quarter-ends"}},
            #     {"curveDefinition": {
            #         "fxCrossCode": "EURGBP"},
            #         "curveParameters": {
            #             "excludeOddPeriods": True,
            #             "excludeLongPeriods": True,
            #             "interpolationMode": "Linear"}
            #     }]
                }
                }).get_data()

<cell>28: error: Unexpected keyword argument "format" for "strftime" of "date"  [call-arg]


In [58]:
deliv_resp.data.raw

{'headers': [{'type': 'String', 'name': 'InstrumentTag'},
  {'type': 'DateTime', 'name': 'ValuationDate'},
  {'type': 'DateTime', 'name': 'StartDate'},
  {'type': 'DateTime', 'name': 'EndDate'},
  {'type': 'String', 'name': 'Tenor'},
  {'type': 'DateTime', 'name': 'NonAdjustedStartDate'},
  {'type': 'DateTime', 'name': 'NonAdjustedEndDate'},
  {'type': 'String', 'name': 'StartDateAdjustmentFlag'},
  {'type': 'String', 'name': 'EndDateAdjustmentFlag'},
  {'type': 'Integer', 'name': 'SpotDecimals'},
  {'type': 'Float', 'name': 'FxSwapPointScalingFactor'},
  {'type': 'Object', 'name': 'StrikeDate_BidAsk'},
  {'type': 'Object', 'name': 'FxSpot_BidMidAsk'},
  {'type': 'Object', 'name': 'FxSpotCcy1_BidMidAsk'},
  {'type': 'Object', 'name': 'FxSpotCcy2_BidMidAsk'},
  {'type': 'Object', 'name': 'FxSwapsCcy1_BidMidAsk'},
  {'type': 'Object', 'name': 'FxSwapsCcy2_BidMidAsk'},
  {'type': 'Object', 'name': 'FxSwapsCcy1Ccy2_BidMidAsk'},
  {'type': 'Object', 'name': 'FxSwapsCcy1Ccy2NearLeg_BidMidAsk

In [59]:
headers_name: list[str] = [h['name'] for h in deliv_resp.data.raw['headers']]
resp_df = pd.DataFrame(data=deliv_resp.data.raw['data'], columns=headers_name)

class dct_str_or_flt(TypedDict):
   bid: float
   ask: float
   mid: float
   instrument: str

i_dct_str_or_flt: dct_str_or_flt
i_int: int
fx_spot_ccy_bid: list[float]
fx_spot_ccy_ask: list[float]
fx_spot_ccy_mid: list[float]
fx_spot_ccy_inst: list[str]
for i_str in ["FxSpotCcy1_BidMidAsk", "FxSpotCcy2_BidMidAsk",
              "FxSwapsCcy1_BidMidAsk", "FxSwapsCcy2_BidMidAsk",
              "FxSwapsCcy1Ccy2_BidMidAsk", "FxSwapsCcy1Ccy2NearLeg_BidMidAsk",
              "FxSwapsCcy1Ccy2FarLeg_BidMidAsk", "FxOutrightCcy1Ccy2_BidMidAsk"]:
    for i_int, i_dct_str_or_flt in enumerate(resp_df[i_str]):
        if len(resp_df[i_str].dropna()) == len(resp_df):
            # if 'instrument' in resp_df[i_str].iloc[i_int].keys():
            if i_int == 0:
                fx_spot_ccy_bid = [i_dct_str_or_flt['bid']]
                fx_spot_ccy_ask = [i_dct_str_or_flt['ask']]
                fx_spot_ccy_mid = [i_dct_str_or_flt['mid']]
                if 'instrument' in resp_df[i_str].iloc[i_int].keys():
                    fx_spot_ccy_inst = [i_dct_str_or_flt['instrument']]
            else:
                fx_spot_ccy_bid.append(i_dct_str_or_flt['bid'])
                fx_spot_ccy_ask.append(i_dct_str_or_flt['ask'])
                fx_spot_ccy_mid.append(i_dct_str_or_flt['mid'])
                if 'instrument' in resp_df[i_str].iloc[i_int].keys():
                    fx_spot_ccy_inst.append(i_dct_str_or_flt['instrument'])
    resp_df[f'{i_str}Bid'.replace("BidMidAsk", "")] = fx_spot_ccy_bid
    resp_df[f'{i_str}Ask'.replace("BidMidAsk", "")] = fx_spot_ccy_ask
    resp_df[f'{i_str}Mid'.replace("BidMidAsk", "")] = fx_spot_ccy_mid
    if len(resp_df[i_str].dropna()) == len(resp_df):
        if 'instrument' in resp_df[i_str].iloc[0].keys():
            resp_df[f'{i_str}Inst'.replace("BidMidAsk", "")] = fx_spot_ccy_inst
resp_df["BidSpot_Calcultd"]=resp_df["FxOutrightCcy1Ccy2_Bid"]-(resp_df["FxSwapsCcy1Ccy2_Bid"]/10000).iloc[2:]
resp_df["AskSpot_Calcultd"]=resp_df["FxOutrightCcy1Ccy2_Ask"]-(resp_df["FxSwapsCcy1Ccy2_Ask"]/10000).iloc[2:]

In [60]:
resp_df[["Tenor", "StartDate", "EndDate", "EndDateAdjustmentFlag", "BidSpot_Calcultd", "AskSpot_Calcultd",
         "FxSwapsCcy1Ccy2_Bid", "FxSwapsCcy1Ccy2_Ask", "FxOutrightCcy1Ccy2_Bid", "FxOutrightCcy1Ccy2_Ask"]]

,Tenor,StartDate,EndDate,EndDateAdjustmentFlag,BidSpot_Calcultd,AskSpot_Calcultd,FxSwapsCcy1Ccy2_Bid,FxSwapsCcy1Ccy2_Ask,FxOutrightCcy1Ccy2_Bid,FxOutrightCcy1Ccy2_Ask
0,ON,2023-11-10T00:00:00Z,2023-11-13T00:00:00Z,W,NaN,NaN,0.886686,0.910019,0.874098,0.874673
1,TN,2023-11-13T00:00:00Z,2023-11-14T00:00:00Z,,NaN,NaN,0.284841,0.306680,0.874189,0.874762
2,SN,2023-11-14T00:00:00Z,2023-11-15T00:00:00Z,,0.87422,0.87479,0.283652,0.305416,0.874248,0.874821
3,SW,2023-11-14T00:00:00Z,2023-11-21T00:00:00Z,,0.87422,0.87479,1.936706,2.168463,0.874414,0.875007
4,1M,2023-11-14T00:00:00Z,2023-12-14T00:00:00Z,,0.87422,0.87479,8.702649,9.217376,0.875090,0.875712
5,6M,2023-11-14T00:00:00Z,2024-05-14T00:00:00Z,,0.87422,0.87479,60.039482,61.960974,0.880224,0.880986
6,9M,2023-11-14T00:00:00Z,2024-08-14T00:00:00Z,,0.87422,0.87479,92.115443,95.695971,0.883432,0.884360
7,1Y,2023-11-14T00:00:00Z,2024-11-14T00:00:00Z,,0.87422,0.87479,127.368153,130.328633,0.886957,0.887823
8,2Y,2023-11-14T00:00:00Z,2025-11-14T00:00:00Z,,0.87422,0.87479,263.237011,275.571349,0.900544,0.902347


In [61]:
# resp_df.to_excel("output6.xlsx")

##### [IPA Curves Service: Construction of the FX Forward Curve (Forward Curves Endpoint)](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-fx-forward-curve)

We saw above how one could collect nessesary data to build FX curves, but it required deep knowlage about the API, the Endpoint, and several of its parameters. The Forward Curves Endpoint allows you to create such FX curves with little such knowlage.

You can find "Manuals & Guides" on "IPA Curves Service: Construction of the FX Forward Curve" [here](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-fx-forward-curve). This set of functionalities is linked with the "https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/forward-curves" Endpoint.

###### Delivery Layer

In [71]:
frwd_crv_deliv_req: rd.delivery._data._endpoint_definition.Definition = rd.delivery.endpoint_request.Definition(
    method=rd.delivery.endpoint_request.RequestMethod.POST,
    url="https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/forward-curves",
    body_parameters={"universe": [
      {"curveDefinition": {
        "baseCurrency": "EUR", # Mandatory
        "quotedCurrency": "GBP", # Mandatory
        # "pivotCurrency": "USD"
        },
       "forwardDefinitions": [{
        "startTenor": "0D"}] # Mandatory
      }]}).get_data()
err_msg: str
if frwd_crv_deliv_req.is_success:
  try:
    frwd_crv_deliv_df: pd.DataFrame = pd.json_normalize(
      frwd_crv_deliv_req.data.raw['data'][0]['forwardCurves'][0]['curve']['curvePoints'])
    frwd_crv_deliv_df.columns.name=frwd_crv_deliv_req.data.raw['data'][0]['curveDefinition']['crossCurrencyDefinitions'][0]['name']
    print("curveDefinition")
    display(frwd_crv_deliv_req.data.raw['data'][0]['curveDefinition'])
    print("curveParameters")
    display(frwd_crv_deliv_req.data.raw['data'][0]['curveParameters'])
    display(frwd_crv_deliv_df)
  except:
    try:
      err_msg = frwd_crv_deliv_req.data.raw['data'][0]['error']
      print("Error 1:")
      display(err_msg)
    except:
      err_msg = frwd_crv_deliv_req.data.raw['data'][0]['forwardCurves'][0]['error']
      print("Error 2:")
      display(err_msg)
else:
  print("Error 3:")
  display(frwd_crv_deliv_req.errors)

curveDefinition


{'baseCurrency': 'EUR',
 'quotedCurrency': 'GBP',
 'crossCurrencyDefinitions': [{'baseCurrency': 'EUR',
   'baseIndexName': 'EURIBOR',
   'name': 'EUR GBP FxForward',
   'quotedCurrency': 'GBP',
   'quotedIndexName': 'LIBOR',
   'source': 'Refinitiv',
   'isNonDeliverable': False,
   'mainConstituentAssetClass': 'FxForward',
   'riskType': 'CrossCurrency',
   'id': 'c2aecf88-3b26-405e-a7f0-30fa2c2918b7',
   'ignoreExistingDefinition': False}]}

curveParameters


{'valuationDate': '2023-11-10',
 'interpolationMode': 'Linear',
 'extrapolationMode': 'Constant',
 'turnAdjustments': {},
 'ignorePivotCurrencyHolidays': False,
 'useDelayedDataIfDenied': False,
 'ignoreInvalidInstrument': True,
 'marketDataLookBack': {'value': 96, 'unit': 'Hour'}}

EUR GBP FxForward,tenor,startDate,endDate,instruments,swapPoint.bid,swapPoint.ask,swapPoint.mid,outright.bid,outright.ask,outright.mid
0,ON,2023-11-10,2023-11-13,[{'instrumentCode': 'EURGBPON='}],0.880000,0.900000,0.890000,0.874280,0.874683,0.874481
1,SPOT,2023-11-14,2023-11-14,NaN,0.000000,0.000000,0.000000,0.874400,0.874800,0.874600
2,SN,2023-11-14,2023-11-15,[{'instrumentCode': 'EURGBPSN='}],0.290000,0.300000,0.295000,0.874429,0.874830,0.874629
3,SW,2023-11-14,2023-11-21,[{'instrumentCode': 'EURGBPSW='}],1.960000,2.200000,2.080000,0.874596,0.875020,0.874808
4,2W,2023-11-14,2023-11-28,[{'instrumentCode': 'EURGBP2W='}],4.100000,4.160000,4.130000,0.874810,0.875216,0.875013
5,3W,2023-11-14,2023-12-05,[{'instrumentCode': 'EURGBP3W='}],6.230000,6.370000,6.300000,0.875023,0.875437,0.875230
6,1M,2023-11-14,2023-12-14,[{'instrumentCode': 'EURGBP1M='}],8.910000,9.000000,8.955000,0.875291,0.875700,0.875495
7,2M,2023-11-14,2024-01-16,[{'instrumentCode': 'EURGBP2M='}],21.505333,21.982000,21.743667,0.876551,0.876998,0.876774
8,3M,2023-11-14,2024-02-14,[{'instrumentCode': 'EURGBP3M='}],30.360000,30.740000,30.550000,0.877436,0.877874,0.877655
9,4M,2023-11-14,2024-03-14,[{'instrumentCode': 'EURGBP4M='}],39.010000,40.530000,39.770000,0.878301,0.878853,0.878577


[Error 400 - invalid_grant] empty error description


###### Content Layer


In [13]:
univ_cont_diff_mat_crvs = forward_curves.Definition(
    curve_definition=forward_curves.SwapZcCurveDefinition(  # This uses the endpoint "https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/forward-curves"
        currency="EUR",
        index_name="EURIBOR",
        discounting_tenor="OIS",
        main_constituent_asset_class="Swap"),
    forward_curve_definitions=[
        forward_curves.ForwardCurveDefinition(
            index_tenor="3M",
            forward_curve_tag="ForwardTag",
            forward_start_date="2023-11-03",
            forward_curve_tenors=["0D", "1D"],
            forward_start_tenor="some_start_tenor")],
    curve_parameters=forward_curves.SwapZcCurveParameters(),
    curve_tag="some_tag").get_data()

<cell>1: error: Name "forward_curves" is not defined  [name-defined]
<cell>2: error: Name "forward_curves" is not defined  [name-defined]
<cell>8: error: Name "forward_curves" is not defined  [name-defined]
<cell>14: error: Name "forward_curves" is not defined  [name-defined]


NameError: name 'forward_curves' is not defined

In [ ]:
univ_cont_diff_mat_crvs.data.df

,endDate,startDate,discountFactor,ratePercent,tenor
0,2023-11-03,2023-11-03,1.0,4.065126,0D
1,2023-11-08,2023-11-03,0.999454,4.065126,1D


In [ ]:


univ_cont_diff_mat_crvs = forward_curves.Definition(
    curve_definition=forward_curves.SwapZcCurveDefinition(
        currency="EUR",
        index_name="EURIBOR",
        discounting_tenor="OIS",
        main_constituent_asset_class="Swap",
    ),
    forward_curve_definitions=[
        forward_curves.ForwardCurveDefinition(
            index_tenor="3M",
            forward_curve_tag="ForwardTag",
            forward_start_date="2021-02-01",
            forward_curve_tenors=["0D", "1D"],
            forward_start_tenor="some_start_tenor",
        )
    ],
    curve_parameters=forward_curves.SwapZcCurveParameters(),
    curve_tag="some_tag",
).get_data()

response.data.df

    instrument_tag="EURGBP",
    fx_cross_type=cross.FxCrossType.FX_FORWARD,
    fx_cross_code="EURGBP",
    fields=["InstrumentTag", "FxSpot_BidMidAsk", "ErrorCode", "Ccy1SpotDate", "Ccy2SpotDate"],
    extended_params = {
        "marketData": {
            "fxSpots": [
                {"spotDefinition": {
                        "fxCrossCode": "AUDUSD", 
                        "Source": "Composite"}}]}}

IndentationError: unexpected indent (4013284653.py, line 23)

Reffering back to the labled EURGBP=R FWDS screenshot above, you can recreate is using the Python API

In [ ]:
rd.close_session()

In [ ]:
https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/forward-curves

In [ ]:
crvs_deliv_req_dict: dict[str, str | dict[str, list[dict[str, dict[str, list[str] | str]]]]] = {
  "endpoint url":"https://api.refinitiv.com/data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/curves",
  "request body": {"universe": [
    {"curveDefinition": {
        "curveTenors": [
        #   "ON", "TN", "SN", "SW", "2W", "3W",
        #   "1M", "2M", "3M", "4M", "5M", "6M", "7M", "8M", "9M", "10M", "11M",
          "1Y", "15M", "18M", "21M", "2Y"],
        # "baseIndexName": "ESTR",
        "baseCurrency": "EUR",
        # "quotedIndexName": "SOFR",
        "quotedCurrency": "GBP"},
      # "curveParameters": {
      #     "valuationDate": "2023-10-06"}
          }]}}

univ_deliv_diff_mat_crvs: rd.delivery._data._endpoint_definition.Definition = rd.delivery.endpoint_request.Definition(
    method=rd.delivery.endpoint_request.RequestMethod.POST,
    url=crvs_req_dict['endpoint url'],
    body_parameters=crvs_req_dict['request body'])
data_deliv_diff_mat_crvs: rd.delivery._data._response.Response = univ_deliv_diff_mat_crvs.get_data()
df_diff_mat_crvs: pd.DataFrame = pd.json_normalize(
    data_deliv_diff_mat_crvs.data.raw['data'][0]['curve']['curvePoints'])

In [ ]:
data_deliv_diff_mat_crvs.data.raw['data'][0]['curveParameters']

{'valuationDate': '2023-11-03',
 'interpolationMode': 'Linear',
 'extrapolationMode': 'Constant',
 'turnAdjustments': {},
 'ignorePivotCurrencyHolidays': False,
 'useDelayedDataIfDenied': False,
 'ignoreInvalidInstrument': True,
 'marketDataLookBack': {'value': 96, 'unit': 'Hour'}}

In [ ]:
data_deliv_diff_mat_crvs.data.raw['data'][0]['curveDefinition']

{'baseCurrency': 'EUR',
 'quotedCurrency': 'GBP',
 'curveTenors': ['1Y', '15M', '18M', '21M', '2Y'],
 'crossCurrencyDefinitions': [{'baseCurrency': 'EUR',
   'baseIndexName': 'EURIBOR',
   'name': 'EUR GBP FxForward',
   'quotedCurrency': 'GBP',
   'quotedIndexName': 'LIBOR',
   'source': 'Refinitiv',
   'isNonDeliverable': False,
   'mainConstituentAssetClass': 'FxForward',
   'riskType': 'CrossCurrency',
   'id': 'c2aecf88-3b26-405e-a7f0-30fa2c2918b7',
   'ignoreExistingDefinition': False}]}

In [ ]:
df_deliv_diff_mat_crvs

,tenor,startDate,endDate,instruments,swapPoint.bid,swapPoint.ask,swapPoint.mid,outright.bid,outright.ask,outright.mid
0,1Y,2023-11-07,2024-11-07,[{'instrumentCode': 'EURGBP1Y='}],133.14,140.40,136.770,0.885014,0.885940,0.885477
1,15M,2023-11-07,2025-02-07,[{'instrumentCode': 'EURGBP15M='}],176.11,178.98,177.545,0.889311,0.889798,0.889555
2,18M,2023-11-07,2025-05-07,[{'instrumentCode': 'EURGBP18M='}],212.20,215.20,213.700,0.892920,0.893420,0.893170
3,21M,2023-11-07,2025-08-07,[{'instrumentCode': 'EURGBP21M='}],249.87,254.48,252.175,0.896687,0.897348,0.897018
4,2Y,2023-11-07,2025-11-07,[{'instrumentCode': 'EURGBP2Y='}],285.75,291.15,288.450,0.900275,0.901015,0.900645


# Conclusion

## Reference of Endpoints' Documentation as of 2023-11-07:

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-fx-forward-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/zc-curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-zero-coupon-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/forward-curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-forward-zero-coupon-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/cross-currency-curves/forward-curves:

This is in progress I will let you know when we create the page.

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/credit-curves/curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-commodity-forward-curve

- /data/quantitative-analytics-curves-and-surfaces/v1/curves/inflation-curves/curves:

https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-curves-service-construction-of-the-inflation-curve